# Calculate the porphyrin dimensions

In [1]:
from math import cos, radians

def average(num_1, num_2 = None):
    if type(num_1) is list:
        return sum(num_1) / len(num_1)
    else:
        if num_2 is not None:
            numbers = [num_1, num_2]
            average = sum(numbers) / len(numbers)
            return average
        else:
            return num_1

# chemical dimensions in Angstroms (as the averages from https://en.wikipedia.org/wiki/Bond_length) and degrees
chemical_dimensions = {
    'bond':{
        'c-c':average(1.2,1.54),
        'c-h':average(1.06,1.12),
        'c-n':average(1.47,2.1),
        'c-f':average(1.34),
        'n=n':average(1.23) # https://doi.org/10.1016/B978-0-08-101033-4.00003-6
    },
    'angle':{
        'sp3':109.5,
        'sp2':120
    }
}

# abbreviate dimensions
cc_bond = chemical_dimensions['bond']['c-c']
cn_bond = chemical_dimensions['bond']['c-n']
mostly_forward_sp2 = cos(radians(chemical_dimensions['angle']['sp2']-90))
less_forward_sp2 = cos(radians(180-chemical_dimensions['angle']['sp2']))

# calculate the total molecular length & width
center_porphyrin_length = (cc_bond*(3*mostly_forward_sp2 + less_forward_sp2))
sp2_extension = (cc_bond * (2 + less_forward_sp2) + cn_bond * (1 + less_forward_sp2))
total_length = sp2_extension + center_porphyrin_length
print(total_length, 'Angstroms')

10.346864409554044 Angstroms


# Define and execute the simulation

In [ ]:
from scipy.constants import micro
from uncertainties import ufloat
%run ../../pdipy/core.py
%matplotlib inline

# define photosensitizer parameters
photosensitizer = {
    'name': '5,10,15,20-tetrakis(1-methylpyridinium-4-yl)porphyrin tetra-iodide',
    'mw': {
        'value':1186.462   # mass with iodides included, sans iodides 678.84,
    },
    'dimensions':{
        'length (A)': total_length,
        'width (A)': total_length,
        'depth (A)': 1.5,
        'shape': 'disc',
    }
}

# define the light parameters
irradiance = 4
light_source = {
    'name': 'OSRAM 2\' 18W/840 lamps',
    "visible_proportion": {
      "value": (700-390)/(700-380),
    }
}


# define general parameters
verbose = True
jupyter = True
bacterial_cfu_ml = 1E8
timestep = 2
total_time = 60
bacterial_specie = 'S_aureus'
photosensitizer_molars = [5*micro, 10*micro, 20*micro]
export_name = 'Beirao_et_al, solution'

# execute the simulation
target_reduction = 1-10**(-7.6)
for photosensitizer_molar in photosensitizer_molars:
    pdi = PDIBacterialPkg(verbose, jupyter)
    pdi.define_system(timestep, total_time, bacterial_cfu_ml = bacterial_cfu_ml)
    pdi.define_bacterium(bacterial_specie)
    pdi.define_photosensitizer(photosensitizer, photosensitizer_molar)
    pdi.define_light(light_source, irradiance)
    pdi.singlet_oxygen_calculations()
    pdi.kinetic_calculation()
    processed_data = pdi.data_processing()
    pdi.export(export_name)
    display(processed_data)

    # affirm the accuracy of the prediction
    value, unit = pdi.data_parsing(target_reduction)    
    if value is not None:
        print(value, unit)
        actual_value = 1
        tolerance = actual_value*0.5
        acceptable_range = [actual_value-tolerance, actual_value+tolerance]
        acceptable_range_float = ufloat(actual_value, tolerance)
        if acceptable_range[0] < value < acceptable_range[1]:
            print(f'The prediction of {value} is within the {acceptable_range_float} tolerance of the actual value.')
        else:
            print(f'--> ERROR: The prediction of {value} is not within the {acceptable_range_float} tolerance of the actual value.')
    else:
        print('--> ERROR: The value was never observed in the simulation predictions.')
    print('\n\n\n\n')

The photosensitizer dimensions as a disc = 1.0346864409554045e-09 m x 1.0346864409554045e-09 m x 1.5e-10 m.
Photosensitizer volume = 1.3E-28 m³
The volume proportion of 7.4E15 photosensitizers = (9.3E-13 m³ of photosensitizer)/(2.4E-6 m³ of solution) = 3.8E-7

('photons per timestep: ', 1.74699162577986e-06)
('molecular oxygen molecules: ', '2.8E-4')
('effective excitation watts: ', '4.3E-3')
// Created by libAntimony v2.12.0.3
model *pdipy_oxidation()

  // Compartments and Species:
  species ps, e_ps, b_ps, mo, so, fa, o_fa;

  // Assignment Rules:
  oxidation := o_fa/(o_fa + fa);

  // Reactions:
  _J0: ps -> e_ps; 20000000000000*5.42633320199086e-5*0.6*ps - 121212121.212121*e_ps;
  _J1: ps => b_ps; 6.96e-10*ps;
  _J2: e_ps + mo => so + ps; 0.48*50000000*e_ps*mo;
  _J3: so => mo; 16666.6666666667*so;
  _J4: so + fa => o_fa + mo; 411.318998088254*so*fa;

  // Species initializations:
  ps = 7.84710694078392e-07;
  e_ps = 4.2152873449902e-06;
  b_ps = 1.96617150066619e-12;
  mo = 0.00

,[ps],[e_ps],[b_ps],[mo],[so],[fa],[ofa]
Time (s),,,,,,,
0.000000,5.000000e-06,0.000000,0.000000e+00,0.000281,0.000000,0.006903,0.000000
124.137931,7.847110e-07,0.000004,6.779903e-14,0.000280,0.000002,0.006330,0.000573
248.275862,7.847110e-07,0.000004,1.355981e-13,0.000280,0.000002,0.005805,0.001098
372.413793,7.847110e-07,0.000004,2.033971e-13,0.000280,0.000002,0.005323,0.001580
496.551724,7.847110e-07,0.000004,2.711961e-13,0.000280,0.000002,0.004881,0.002022
620.689655,7.847109e-07,0.000004,3.389951e-13,0.000280,0.000002,0.004476,0.002427
744.827586,7.847109e-07,0.000004,4.067942e-13,0.000280,0.000002,0.004105,0.002798
868.965517,7.847109e-07,0.000004,4.745932e-13,0.000280,0.000002,0.003764,0.003139
993.103448,7.847109e-07,0.000004,5.423922e-13,0.000280,0.000002,0.003452,0.003451


The raw oxidation data is being converting into a bacterial inactivation predictions. This may take a minute.
